In [22]:
import csv
import cv2
import numpy as np
import sklearn

In [23]:
#read csv data file
lines = []

with open('./data/driving_log.csv') as csvDataFile:
    csvReader = csv.reader(csvDataFile)
    for row in csvReader:
        lines.append(row)

In [24]:
lines[0]

['/Users/manika/Desktop/moreData/IMG/center_2017_10_29_01_23_40_351.jpg',
 '/Users/manika/Desktop/moreData/IMG/left_2017_10_29_01_23_40_351.jpg',
 '/Users/manika/Desktop/moreData/IMG/right_2017_10_29_01_23_40_351.jpg',
 '0',
 '0',
 '0',
 '4.633465E-06']

In [25]:
#read the center images
images = []
measurements = []
for line in lines:
#     for i in range(3):
#         imagePath = line[i]
    sourcePath = line[0]
    fileName = sourcePath.split('/')[-1]
    currentPath = './data/IMG/'+fileName
#     image = cv2.imread(currentPath)
#         image = cv2.imread(imagePath)
#     images.append(image)
    images.append(currentPath)
    measurement = float (line[3])
    measurements.append(measurement)

In [31]:
centerImages = []
leftImages = []
rightImages = []
measurements = []

for line in lines:
    centerPath = line[0]
    leftPath = line[1]
    rightPath = line[2]
    fileNameCenter = centerPath.split('/')[-1]
    fileNameLeft = leftPath.split('/')[-1]
    fileNameRight = leftPath.split('/')[-1]
    centerImages.append('./data/IMG/'+fileNameCenter)
    leftImages.append('./data/IMG/'+fileNameLeft)
    rightImages.append('./data/IMG/'+fileNameRight)
    measurements.append(float (line[3]))

In [ ]:
def combineImages(center, left, right, measurement, correction):
    imagePaths = []
    imagePaths.extend(center)
    imagePaths.extend(left)
    imagePaths.extend(right)
    measurements = []
    measurements.extend(measurement)
    measurements.extend([x + correction for x in measurement])
    measurements.extend([x - correction for x in measurement])
    return (imagePaths, measurements)

In [ ]:
images, measurements = combineImages(centerImages, leftImages, rightImages, measurements, 0.2)

In [26]:
from sklearn.model_selection import train_test_split
samples = list(zip(images, measurements))
train_samples, validation_samples = train_test_split(samples, test_size=0.2)

def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        samples = sklearn.utils.shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for imagePath, measurement in batch_samples:
#                 fileName = imagePath.split('/')[-1]
#                 currentPath = './data/IMG/'+fileName
                image = cv2.imread(imagePath)
                
#                 originalImage = cv2.imread(imagePath)
#                 image = cv2.cvtColor(originalImage, cv2.COLOR_BGR2RGB)
                images.append(image)
                angles.append(measurement)
                # Flipping
                images.append(cv2.flip(image,1))
                angles.append(measurement*-1.0)

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)

In [27]:
# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)

In [28]:
# print (images[0])

In [29]:
# measurements[0]

In [ ]:
#data augmentation
augmentedImages, augmentedMeasurements = [], []
for image, measurement in zip(images, measurements):
    augmentedImages.append(image)
    augmentedMeasurements.append(measurement)
    augmentedImages.append(cv2.flip(image, 1))
    augmentedMeasurements.append(measurement*-1.0)

In [ ]:
#convert the images and measurements to train data and label
# X_train = np.array(images)
# y_train = np.array(measurements)
# X_train = np.array(augmentedImages)
# y_train = np.array(augmentedMeasurements)

In [ ]:
len(X_train)

In [ ]:
# #build a basic neural network to check if everything is working
# from keras.models import Sequential
# from keras.layers import Flatten, Dense, Lambda, Activation, Cropping2D
# from keras.layers.convolutional import Convolution2D
# from keras.layers.convolutional import MaxPooling2D

# #Lenet Architecture
# model = Sequential()
# model.add(Cropping2D(cropping=((50,20), (0,0)), input_shape=(3,160,320)))
# model.add(Lambda(lambda x:x/255.0-0.5, input_shape=(160, 320, 3)))
# model.add(Convolution2D(6, (5, 5)))
# model.add(Activation("relu"))
# model.add(MaxPooling2D())
# model.add(Convolution2D(6, (5, 5)))
# model.add(Activation("relu"))
# model.add(MaxPooling2D())
# model.add(Flatten())
# model.add(Dense(120))
# model.add(Dense(84))
# model.add(Dense(1))

# model.compile(loss='mse', optimizer='adam')
# model.fit(X_train, y_train, validation_split = 0.2, shuffle=True, epochs=4) #nb_epoch is set to 7 as after this the loss increses

# model.save('modelAug.h5')

In [30]:
#NVIDIA architecture

from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Activation, Cropping2D
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D

#Lenet Architecture
model = Sequential()
model.add(Cropping2D(cropping=((70,20), (0,0)), input_shape=(160,320,3)))
model.add(Lambda(lambda x:x/255.0-0.5))
model.add(Convolution2D(24, (5, 5), strides=(2,2), activation='relu'))
model.add(Convolution2D(36, (5, 5), strides=(2,2), activation='relu'))
model.add(Convolution2D(48, (5, 5), strides=(2,2), activation='relu'))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
# model.fit(X_train, y_train, validation_split = 0.2, shuffle=True, epochs=5) #nb_epoch is set to 7 as after this the loss increses
history_object = model.fit_generator(train_generator, samples_per_epoch= \
                 len(train_samples), validation_data=validation_generator, \
                 nb_val_samples=len(validation_samples), epochs=3)

model.save('modelGen.h5')

/Users/manika/anaconda3/envs/behavior/lib/python2.7/site-packages/ipykernel_launcher.py:25: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/Users/manika/anaconda3/envs/behavior/lib/python2.7/site-packages/ipykernel_launcher.py:25: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_data=<generator..., steps_per_epoch=7135, epochs=3, validation_steps=1784)`


Epoch 1/3
7135/7135 [==============================] - 6953s - loss: 0.0163 - val_loss: 0.0448
Epoch 2/3
7135/7135 [==============================] - 6756s - loss: 0.0016 - val_loss: 0.0404
Epoch 3/3
7135/7135 [==============================] - 6768s - loss: 9.9471e-04 - val_loss: 0.0392
